In [1]:
import numpy as np
import pandas as pd

from preparation import prepare_data
from handle_reports import generate_report, append_report
from monitoring import cur_stats
from plotting import plot_data

Here you test something

In [2]:
PATH = "C:/Users/San/Documents/inf/time monitoring/test_data.txt"
DAY_OFFS = []

Cols meaning:
- Day - day of the month
- DKind - day kind
- SIL - sport intensity level - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- Jug - juggling
- GMG - gaming


(Separate message about SIL)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [3]:
# Create list of colnames
colnames = ["Day", "Math", "CS", "Eng", "Sport",
            "SIL", "OD", "LE", "Jug", "GMG"]
# Load data into df with sep equal to ':' and '_'
df = pd.read_csv(
    filepath_or_buffer=PATH, sep="[:_-]", names=colnames,
    header=None, engine="python")
# Create var 'month' with df data from row 0 and col 0
month = df.iloc[0][0]
# Drop the row 0 where 'month' was
df.drop(index=0, inplace=True)
# Reset index, so it again starts with 0
df = df.reset_index(drop=True)
df

,Day,Math,CS,Eng,Sport,SIL,OD,LE,Jug,GMG
0,17,90.0,85.0,131.0,pa9,3.0,ou60,le5,ju0,ga240
1,18,80.0,110.0,70.0,pa10,3.0,ou60,le5,ju0,ga65
2,19,70.0,90.0,65.0,pa10,0.0,ou70,le0,ju10,ga260
3,20,70.0,70.0,35.0,pa11,3.0,ou40,le5,ju0,ga150


In [ ]:
def sport_stats():
    """
    (Example)
    You exercised this month: 10 times
    Last time you got medium exercise: 3 days ago
    Last time you got hard exercise: 7 days ago
    """
    pass

In [4]:
# possible new name for modified prepare_data func
def prepare_df():
    pass

In [5]:
def handle_day_offs(DAY_OFFS):
    # Create two day_offs vars for generate_report
    # func. With these two vars, reports will
    # provide a more detailed picture of the month
    day_offs_count = len(DAY_OFFS)
    if day_offs_count > 0:
        day_offs_str = ', '.join(map(str, DAY_OFFS))
    else:
        day_offs_str = "you studied every day"
    return day_offs_count, day_offs_str

# encapsulate day_offs_count, day_offs_str into report_data
# variable somehow
day_offs_count, day_offs_str = handle_day_offs(DAY_OFFS)

In [6]:
# small func used in lambda expression
def categorise(row, DAY_OFFS):
    if row["Day"] in DAY_OFFS:
        return "relax"
    return "work"

# in the cols below first two symbols are for easier
# identification while writing the data. Many thanks
# to https://stackoverflow.com/a/42349635/11749578
cols_to_shorten = ["Sport", "OD", "LE", "Jug", "GMG"]
for col in cols_to_shorten:
    df[col] = df[col].str[2:]
# Convert col dtypes to int64
df[colnames] = df[colnames].astype("int64")
# Create a new col with info about whether
# you work or relax on a particular day
df["DKind"] = df.apply(lambda row: categorise(row, DAY_OFFS), axis=1)
# Rearrange order of cols
cols = df.columns.tolist()
cols = [cols[0]] + [cols[-1]] + cols[1:-1]


In [7]:
df = df[cols]

In [8]:
df

,Day,DKind,Math,CS,Eng,Sport,SIL,OD,LE,Jug,GMG
0,17,work,90,85,131,9,3,60,5,0,240
1,18,work,80,110,70,10,3,60,5,0,65
2,19,work,70,90,65,10,0,70,0,10,260
3,20,work,70,70,35,11,3,40,5,0,150


In [9]:
df.dtypes

Day       int64
DKind    object
Math      int64
CS        int64
Eng       int64
Sport     int64
SIL       int64
OD        int64
LE        int64
Jug       int64
GMG       int64
dtype: object

In [10]:
def calculate_study_data(df, DESIRED_MEAN_VALUE):
    """
    Summary:
        complete all the study data calculations 
        in the program. Study data cols are:
        Day, DKind, Math, CS, and Eng
        (Day and DKind are used to identify
        workdays and day offs)
    Args:
        df: pandas dataframe
    Returns:
        list of data
    """
    # get summed values from 'Math', 'CS', and 'Eng' cols
    total_per_subject = df[["Math", "CS", "Eng"]].sum(
        axis=0).div(60).round().astype(np.int64)
    math_hs, cs_hs, eng_hs = total_per_subject
    # create col 'Total' where each row is the sum
    # of 'Math', 'CS', and 'Eng' cols
    total_per_day = df[["Math", "CS", "Eng"]].sum(axis=1)
    df["Total"] = total_per_day
    # with full data mean it's easier to compare
    # the change of your study time because
    # number of day offs per months differ
    # from month to month
    mean_full_data = round(total_per_day.mean())
    # find how many hours you studied this month
    total_hs = round(total_per_day.sum() / 60)
    # handle vacation days. Your mean and std
    # must not include data from vacation days.
    if "relax" in df["DKind"].values:
        df_removed_day_offs = df[~df["DKind"].str.contains("relax")]
        total_per_day = df_removed_day_offs["Total"]
    mean = round(total_per_day.mean())
    std = round(total_per_day.std(ddof=0))
    min_to_study = calc_req_study_time(total_per_day, DESIRED_MEAN_VALUE)
    return [df, math_hs, cs_hs, eng_hs, total_hs,
            mean_full_data, mean, std, min_to_study]
    
# small func to calculate how many more min you need to
# study to achieve your desired monthly mean
def calc_req_study_time(total_per_day, DESIRED_MEAN_VALUE):
    req_min = DESIRED_MEAN_VALUE * len(total_per_day)
    studied_min = sum(total_per_day)
    return req_min - studied_min


In [11]:
df_study, math_hs, cs_hs, eng_hs, total_hs, mean_full_data, mean, std, min_to_study = calculate_study_data(df.iloc[:, :5], 240)
df_study # combine here this df with the other cols (sport, sil, and so on)


,Day,DKind,Math,CS,Eng,Total
0,17,work,90,85,131,306
1,18,work,80,110,70,260
2,19,work,70,90,65,225
3,20,work,70,70,35,175


In [12]:
df # here must be a df with col 'Total' after 'Eng' col instead

,Day,DKind,Math,CS,Eng,Sport,SIL,OD,LE,Jug,GMG
0,17,work,90,85,131,9,3,60,5,0,240
1,18,work,80,110,70,10,3,60,5,0,65
2,19,work,70,90,65,10,0,70,0,10,260
3,20,work,70,70,35,11,3,40,5,0,150
